In [1]:
FN = 'train'

In [2]:
import os 
os.environ['KERAS_BACKEND'] = 'tensorflow'
import keras

import pickle
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys
import Levenshtein
from sklearn.model_selection import train_test_split
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [3]:
FN0 = 'vocabulary-embedding'

In [4]:
FN1 = 'train'

In [5]:
maxlend=25 # 0 - if we dont want to use description at all
maxlenh=25
maxlen = maxlend + maxlenh
rnn_size = 512 # must be same as 160330-word-gen
rnn_layers = 3  # match FN1
batch_norm=False

In [6]:
activation_rnn_size = 40 if maxlend else 0

In [7]:
# training parameters
seed=42
p_W, p_U, p_dense, p_emb, weight_decay = 0, 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size=64
nflips=10

In [8]:
nb_train_samples = 1000
nb_val_samples = 100

In [10]:
with open('data/%s.pkl'%FN0, 'rb') as fp:
    embedding, idx2word, word2idx, glove_idx2idx = pickle.load(fp)

In [19]:
embedding = embedding[0:10000]

In [20]:
vocab_size, embedding_size = embedding.shape

In [23]:
with open('data/%s.data1.pkl'%FN0, 'rb') as fp:
    X, Y = pickle.load(fp)

In [24]:
len(X)

200001

In [25]:
X = X[0:20000]

In [26]:
Y = Y[0:20000]

In [27]:
nb_unknown_words = 10

In [28]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

In [29]:
oov0 = vocab_size-nb_unknown_words

In [30]:
for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i]+'^'

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

(19900, 19900, 100, 100)

In [32]:
del X
del Y

In [33]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [34]:
def prt(label, x):
    print(label+':',end=" ")
    for w in x:
        print(idx2word[w],end=" ")
    print("")

In [35]:
i = 334
prt('H',Y_train[i])
prt('D',X_train[i])

H: The Wall Street Journal Hey^ Mr. DJ Technics^ turntables^ are coming back 
D: Responding^ to longtime calls by fans and disk^ jockeys,^ Panasonic^ Corp. said it is reviving^ Technics^ turntables^ -- the series that remains a de facto^ standard player supporting nightclub^ music scenes.^ The new analog^ turntable,^ which would come with new direct-drive^ motor technologies that Panasonic^ says would improve the quality of sound,^ would be released sometime^ between April 2016 and March 2017,^ the Japanese electronics company announced on Wednesday. Panasonic^ last year relaunched^ the high-end music brand that was discontinued^ in 2010 as a part of the Japanese company's strategy to unif...^ 


In [39]:
i = 34
prt('H',Y_test[i])
prt('D',X_test[i])

H: When College Expenses^ Are a Family Affair^ 
D: Contributed^ News: ETFGI:^ Global ETFs/ETPs^ Gather^ Assets^ During Roller^ Coaster^ Month RIA^ Firms^ that Offer Financial Planning are Surviving^ and Thriving^ Why You Should Introduce^ Your Adult^ Children to Your Financial Advisor As a grandparent,^ you probably take a certain amount of pleasure in showering^ your grandkids^ with the occasional^ extravagance.^ That takes me to a story shared by my colleague^ Michael Graci,^ an investor education and retirement specialist here at BlackRock.^ Mike’s^ sister was contemplating^ the idea of buying her nine-year-old^ granddaughter^ a pony^ for her birthday.^ Mike tallied^ the potential cost in his head (horse,^ boarding^ fees,^ veterinary^ bills,^ feed,^ saddle,^ bridles^ and, of course, riding lessons).^ Consummate^ professional (and good brother)^ that he is, Mike suggested she take that pony^ money and put it into a 529^ plan for her granddaughter’s^ education. Tax-free^ growth over t

Model

In [40]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.initializers import Constant

In [41]:
# seed weight initialization
random.seed(seed)
np.random.seed(seed)

In [42]:
regularizer = l2(weight_decay) if weight_decay else None

In [43]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    embeddings_regularizer=regularizer, trainable=False, embeddings_initializer=Constant(embedding), mask_zero=True,
                    name='embedding_1'))
for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True,name='lstm_%d'%(i+1))
    model.add(lstm)
    model.add(Dropout(p_dense,name='dropout_%d'%(i+1)))

In [44]:
from keras.layers.core import Lambda
import keras.backend as K

def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:,:maxlend,:], X[:,maxlend:,:]
    head_activations, head_words = head[:,:,:n], head[:,:,n:]
    desc_activations, desc_words = desc[:,:,:n], desc[:,:,n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    # make sure we dont use description words that are masked out
    activation_energies = activation_energies + -1e20*K.expand_dims(1.-K.cast(mask[:, :maxlend],'float32'),1)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    # for every head word compute weighted average of desc words
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=(2,1))
    return K.concatenate((desc_avg_word, head_words))


class SimpleContext(Lambda):
    def __init__(self,**kwargs):
        super(SimpleContext, self).__init__(simple_context,**kwargs)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def compute_output_shape(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [45]:
if activation_rnn_size:
    model.add(SimpleContext(name='simplecontext_1'))
model.add(TimeDistributed(Dense(vocab_size,name = 'timedistributed_1')))
model.add(Activation('softmax', name='activation_1'))

In [46]:
from keras.optimizers import Adam, RMSprop # usually I prefer Adam but article used rmsprop
# opt = Adam(lr=LR)  # keep calm and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)



In [47]:
%%javascript
//new Audio("http://www.soundjay.com/button/beep-09.wav").play ()

<IPython.core.display.Javascript object>

In [48]:
K.set_value(model.optimizer.lr,np.float32(LR))

In [49]:
def str_shape(x):
    return 'x'.join(map(str,x.shape))
    
def inspect_model(model):
    for i,l in enumerate(model.layers):
        print(i, 'cls=%s name=%s'%(type(l).__name__, l.name))
        weights = l.get_weights()
        for weight in weights:
            print(str_shape(weight),end=" ")
        print("")

In [50]:
inspect_model(model)


0 cls=Embedding name=embedding_1
10000x100 
1 cls=LSTM name=lstm_1
100x2048 512x2048 2048 
2 cls=Dropout name=dropout_1

3 cls=LSTM name=lstm_2
512x2048 512x2048 2048 
4 cls=Dropout name=dropout_2

5 cls=LSTM name=lstm_3
512x2048 512x2048 2048 
6 cls=Dropout name=dropout_3

7 cls=SimpleContext name=simplecontext_1

8 cls=TimeDistributed name=time_distributed_1
944x10000 10000 
9 cls=Activation name=activation_1



In [51]:
if FN1 and os.path.exists('data/%s.hdf5'%FN1):
    model.load_weights('data/%s.hdf5'%FN1)

In [52]:
def lpadd(x, maxlend=maxlend, eos=eos):
    """left (pre) pad a description to maxlend and then add eos.
    The eos is the input to predicting the first word in the headline
    """
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty]*(maxlend-n) + x + [eos]

In [53]:
samples = [lpadd([3]*26)]
# pad from right (post) so the first maxlend will be description followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')

In [54]:
np.all(data[:,maxlend] == eos)

True

In [55]:
data.shape,map(len, samples)

((1, 50), <map at 0x7f8cdc91b7f0>)

In [56]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

(1, 25, 10000)

In [104]:
def beamsearch(predict, start=[empty]*maxlend + [eos],
               k=1, maxsample=maxlen, use_unk=True, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in range(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_k = 0 # samples that reached eos
    dead_samples = []
    dead_scores = []
    live_k = 1 # samples that did not yet reached eos
    live_samples = [list(start)]
    live_scores = [0]

    while live_k:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk:
            for i in range(nb_unknown_words):
                cand_scores[:,vocab_size - 1 - i] = 1e20
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        ranks_dead = [r for r in ranks if r < n]
        ranks_live = [r - n for r in ranks if r >= n]
        
        dead_scores = [dead_scores[r] for r in ranks_dead]
        dead_samples = [dead_samples[r] for r in ranks_dead]
        
        live_scores = [live_scores[r] for r in ranks_live]

        # append the new words to their appropriate live sample
        voc_size = probs.shape[1]
        live_samples = [live_samples[r//voc_size]+[r%voc_size] for r in ranks_live]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        zombie = [s[-1] == eos or len(s) > maxsample for s in live_samples]
        
        # add zombies to the dead
        dead_samples += [s for s,z in zip(live_samples,zombie) if z]
        dead_scores += [s for s,z in zip(live_scores,zombie) if z]
        dead_k = len(dead_samples)
        # remove zombies from the living 
        live_samples = [s for s,z in zip(live_samples,zombie) if not z]
        live_scores = [s for s,z in zip(live_scores,zombie) if not z]
        live_k = len(live_samples)

    return dead_samples + live_samples, dead_scores + live_scores

In [88]:
qwer = [[1,2,3],[4,5,6]]
qwer[:,0]

TypeError: list indices must be integers or slices, not tuple

In [100]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
#     sample_lengths = map(len, samples)
    sample_lengths = []
    for i in samples:
        sample_lengths.append(len(i))
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([prob[sample_length-maxlend-1] for prob, sample_length in zip(probs, sample_lengths)])

In [59]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < oov0 else glove_idx2idx.get(x,x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= oov0])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x,vocab_size-1-min(i, nb_unknown_words-1)) for i, x in enumerate(outside))
    xs = [outside.get(x,x) for x in xs]
    return xs

In [60]:
def vocab_unfold(desc,xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= oov0:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x,x) for x in xs]

In [61]:
def gensamples(skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    i = random.randint(0,len(X_test)-1)
    print('HEAD:',' '.join(idx2word[w] for w in Y_test[i][:maxlenh]))
    print('DESC:',' '.join(idx2word[w] for w in X_test[i][:maxlend]))
    sys.stdout.flush()

    print('HEADS:')
    x = X_test[i]
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend,len(x)), max(maxlend,len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s,start,scr) for s,scr in zip(sample,score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code,c) for c in codes])
            if distance > -0.6:
                print(score, ' '.join(words))
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print(score, ' '.join(words))
        codes.append(code)

In [105]:
gensamples(skips=2, batch_size=batch_size, k=10, temperature=1.)

HEAD: Make Your Own^ Cap^ Toe^ Flats^ – DIY
DESC: Yes, we all admit that ballet^ flats^ are among the cutest^ and most versatile^ shoes ever!^ Team them up with a dainty^ skirt,^ a pair
HEADS:
3.8325839042663574 
22.99684500694275 ballet^ fall flats^
23.57991123199463 at need:^ court


In [63]:
def flip_headline(x, nflips=None, model=None, debug=False):
    """given a vectorized input (after `pad_sequences`) flip some of the words in the second half (headline)
    with words predicted by the model
    """
    if nflips is None or model is None or nflips <= 0:
        return x
    
    batch_size = len(x)
    assert np.all(x[:,maxlend] == eos)
    probs = model.predict(x, verbose=0, batch_size=batch_size)
    x_out = x.copy()
    for b in range(batch_size):
        # pick locations we want to flip
        # 0...maxlend-1 are descriptions and should be fixed
        # maxlend is eos and should be fixed
        flips = sorted(random.sample(range(maxlend+1,maxlen), nflips))
        if debug and b < debug:
            print(b,end=" ")
        for input_idx in flips:
            if x[b,input_idx] == empty or x[b,input_idx] == eos:
                continue
            # convert from input location to label location
            # the output at maxlend (when input is eos) is feed as input at maxlend+1
            label_idx = input_idx - (maxlend+1)
            prob = probs[b, label_idx]
            w = prob.argmax()
            if w == empty:  # replace accidental empty with oov
                w = oov0
            if debug and b < debug:
                print('%s => %s'%(idx2word[x_out[b,input_idx]],idx2word[w]),end=" ")
            x_out[b,input_idx] = w
        if debug and b < debug:
            print("")
    return x_out

In [64]:
def conv_seq_labels(xds, xhs, nflips=None, model=None, debug=False):
    """description and hedlines are converted to padded input vectors. headlines are one-hot to label"""
    batch_size = len(xhs)
    assert len(xds) == batch_size
    x = [vocab_fold(lpadd(xd)+xh) for xd,xh in zip(xds,xhs)]  # the input does not have 2nd eos
    x = sequence.pad_sequences(x, maxlen=maxlen, value=empty, padding='post', truncating='post')
    x = flip_headline(x, nflips=nflips, model=model, debug=debug)
    
    y = np.zeros((batch_size, maxlenh, vocab_size))
    for i, xh in enumerate(xhs):
        xh = vocab_fold(xh) + [eos] + [empty]*maxlenh  # output does have a eos at end
        xh = xh[:maxlenh]
        y[i,:,:] = np_utils.to_categorical(xh, vocab_size)
        
    return x, y

In [65]:
def gen(Xd, Xh, batch_size=batch_size, nb_batches=None, nflips=None, model=None, debug=False, seed=seed):
    """yield batches. for training use nb_batches=None
    for validation generate deterministic results repeating every nb_batches
    
    while training it is good idea to flip once in a while the values of the headlines from the
    value taken from Xh to value generated by the model.
    """
    c = nb_batches if nb_batches else 0
    while True:
        xds = []
        xhs = []
        if nb_batches and c >= nb_batches:
            c = 0
        new_seed = random.randint(0, sys.maxsize)
        random.seed(c+123456789+seed)
        for b in range(batch_size):
            t = random.randint(0,len(Xd)-1)

            xd = Xd[t]
            s = random.randint(min(maxlend,len(xd)), max(maxlend,len(xd)))
            xds.append(xd[:s])
            
            xh = Xh[t]
            s = random.randint(min(maxlenh,len(xh)), max(maxlenh,len(xh)))
            xhs.append(xh[:s])

        # undo the seeding before we yield inorder not to affect the caller
        c+= 1
        random.seed(new_seed)

        yield conv_seq_labels(xds, xhs, nflips=nflips, model=model, debug=debug)

In [66]:
r = next(gen(X_train, Y_train, batch_size=batch_size))
r[0].shape, r[1].shape, len(r)

((64, 50), (64, 25, 10000), 2)

In [67]:
def test_gen(gen, n=5):
    Xtr,Ytr = next(gen)
    for i in range(n):
        assert Xtr[i,maxlend] == eos
        x = Xtr[i,:maxlend]
        y = Xtr[i,maxlend:]
        yy = Ytr[i,:]
        yy = np.where(yy)[1]
        prt('L',yy)
        prt('H',y)
        if maxlend:
            prt('D',x)

In [68]:
test_gen(gen(X_train, Y_train, batch_size=batch_size))

L: <1>^ New Jersey Investments Continue to Create Local <2>^ to <0>^ More and Put More People to Work ~ _ _ _ _ _ _ 
H: ~ <6>^ New Jersey Investments Continue to Create Local <7>^ to <2>^ More and Put More People to Work _ _ _ _ _ _ 
D: combined cycle power plant on its <5>^ generating Station site in <4>^ New <1>^ The <0>^ which represents an investment of more than <3>^ million, 
L: <4>^ <3>^ Out That This Girl <1>^ <0>^ <2>^ Or jumping Into the Water Is The New <5>^ ~ _ _ _ _ _ _ 
H: ~ <7>^ <5>^ Out That This Girl <2>^ <0>^ <3>^ Or jumping Into the Water Is The New <9>^ _ _ _ _ _ _ 
D: or <9>^ depending on who you <6>^ <8>^ user <9>^ <1>^ the photo on Thursday night, and it <4>^ to more than two million views 
L: <1>^ <0>^ start strong for networks ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ <4>^ <3>^ start strong for networks _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: <5>^ million. ___ ABC is owned by The <0>^ Disney Co. CBS is owned by CBS Corp. <2>^ is a joint venture of Warner <1>^

In [69]:
test_gen(gen(X_train, Y_train, nflips=6, model=model, debug=False, batch_size=batch_size))

L: <1>^ New Jersey Investments Continue to Create Local <2>^ to <0>^ More and Put More People to Work ~ _ _ _ _ _ _ 
H: ~ <6>^ New Jersey Investments Continue to cool Local <7>^ to <2>^ More and cool cool People to Work _ _ _ _ _ _ 
D: combined cycle power plant on its <5>^ generating Station site in <4>^ New <1>^ The <0>^ which represents an investment of more than <3>^ million, 
L: <4>^ <3>^ Out That This Girl <1>^ <0>^ <2>^ Or jumping Into the Water Is The New <5>^ ~ _ _ _ _ _ _ 
H: ~ <7>^ <5>^ Out changed. This Girl <2>^ equipment, Notre Or jumping Notre the Water Notre The him, <9>^ _ _ _ _ _ _ 
D: or <9>^ depending on who you <6>^ <8>^ user <9>^ <1>^ the photo on Thursday night, and it <4>^ to more than two million views 
L: <1>^ <0>^ start strong for networks ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ <4>^ Iran start strong for networks _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: <5>^ million. ___ ABC is owned by The <0>^ Disney Co. CBS is owned by CBS Corp. <2>^ is a joint venture

In [70]:
valgen = gen(X_test, Y_test,nb_batches=3, batch_size=batch_size)

In [71]:
for i in range(4):
    test_gen(valgen, n=1)

L: High school girls <4>^ <5>^ <0>^ <3>^ <2>^ in second leg of county <1>^ ~ _ _ _ _ _ _ _ _ _ _ 
H: ~ High school girls <6>^ <8>^ <1>^ <5>^ <3>^ in second leg of county <2>^ _ _ _ _ _ _ _ _ _ _ 
D: piece and <0>^ a late penalty <4>^ <8>^ has a week to prepare for <9>^ then hosts Park City in a match that will likely 
L: New regulations for marijuana ahead ~ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ New regulations for marijuana ahead _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: However, all around the <1>^ <0>^ this week, at events organized to celebrate the starting harvest season, local cannabis <2>^ industry members, and regulators discussed 
L: <0>^ <1>^ and Linda <3>^ Hope to Act as a <2>^ ~ _ _ _ _ _ _ _ _ _ _ _ _ _ 
H: ~ <1>^ <3>^ and Linda <5>^ Hope to Act as a <4>^ _ _ _ _ _ _ _ _ _ _ _ _ _ 
D: qualified to compete for <6>^ <7>^ <1>^ admitted to not being clear about why there was a change in the <2>^ and also did not 
L: High school girls <4>^ <5>^ <0>^ <3>^ <2>^ in second le

Train

In [72]:
history = {}

In [73]:
traingen = gen(X_train, Y_train, batch_size=batch_size, nflips=nflips, model=model)
valgen = gen(X_test, Y_test, nb_batches=nb_val_samples//batch_size, batch_size=batch_size)

### r = next(traingen)
r[0].shape, r[1].shape, len(r)

In [111]:
for iteration in range(500):
    print('Iteration', iteration)
    h = model.fit_generator(traingen, steps_per_epoch=((nb_train_samples)/batch_size),
                        epochs=1, validation_data=valgen, validation_steps=(nb_val_samples/batch_size)
                           )
    for k,v in h.history.items():
        history[k] = history.get(k,[]) + v
    with open('data/%s.history.pkl'%FN,'wb') as fp:
        pickle.dump(history,fp,-1)
    model.save_weights('data/%s.hdf5'%FN, overwrite=True)
    gensamples(batch_size=batch_size)
    

Iteration 0
Epoch 1/1
16/15 [==============================] - 10s 642ms/step - loss: 5.9523 - val_loss: 6.0124
HEAD: UK must do more to help refugees, Nicola^ Sturgeon^ says
DESC: Nicola^ Sturgeon^ has insisted the UK can and must do more to help refugees fleeing Syria for Europe. Scotland should accept 1,000 people as a
HEADS:
2.3744280338287354 
5.031222343444824 Sturgeon^
6.217995643615723 <4>^
